In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Đọc dữ liệu từ file CSV
data = pd.read_csv("NLP DATASET.csv") 
print(data.head())


   STT                                           Question     Label
0    1                    Vì sao mèo ăn những vật thể lạ?       Why
1    2      Bạn đã chạy được bao nhiêu km trong tuần này?  Quantity
2    3  Tôi tính làm tủ áo cánh kính cửa lùa, có nên l...    Yes/No
3    4  Cách phòng ngừa và điều trị Búi lông “Hairball...       How
4    5  Bạn đã tham gia bao nhiêu buổi tư vấn nghề ngh...  Quantity


In [31]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
def tokenize_data(texts, tokenizer, max_length=128):
    input_ids_list = []
    attention_masks_list = []

    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            max_length=max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True
        )
        input_ids_list.append(encoded['input_ids'])
        attention_masks_list.append(encoded['attention_mask'])

    return input_ids_list, attention_masks_list

# Mã hóa câu hỏi
input_ids, attention_masks = tokenize_data(data['Question'], tokenizer)

# Tạo DataFrame đã xử lý
processed_data = pd.DataFrame({
    'Question': data['Question'],   # Câu hỏi gốc
    'Label': data['Label'],         # Nhãn
    'Input_Ids': [str(ids) for ids in input_ids],  # Mã hóa input_ids
    'Attention_Mask': [str(mask) for mask in attention_masks]  # Mã hóa attention_mask
})

# Lưu processed data
processed_data.to_csv("Data/Processed_Data/transformer_processed_dataset.csv", index=False)

In [32]:
# Chia dữ liệu thành train, test, val
X = data['Question']
y = data['Label']
# Chia dữ liệu thành train (80%) và test (20%)
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chia tiếp train_full thành train (80%) và validation (20%)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# Lưu từng tập vào file CSV
train_data = pd.DataFrame({'Question': X_train, 'Label': y_train})
val_data = pd.DataFrame({'Question': X_val, 'Label': y_val})
test_data = pd.DataFrame({'Question': X_test, 'Label': y_test})

train_data.to_csv("Data/Train_Test_Val/train_data.csv", index=False)
val_data.to_csv("Data/Train_Test_Val/val_data.csv", index=False)
test_data.to_csv("Data/Train_Test_Val/test_data.csv", index=False)


In [33]:
from torch.utils.data import DataLoader, Dataset
import torch
from sklearn.preprocessing import LabelEncoder

# Mã hóa nhãn
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train).tolist()  # Chuyển sang list
y_val_encoded = label_encoder.transform(y_val).tolist()  # Chuyển sang list
y_test_encoded = label_encoder.transform(y_test).tolist()  # Chuyển sang list

# Tạo CustomDataset
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts.tolist() 
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]  # Truy cập bằng index
        label = self.labels[idx]  

        encoded = self.tokenizer.encode_plus(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            'input_ids': encoded['input_ids'].squeeze(0),
            'attention_mask': encoded['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

train_dataset = CustomDataset(X_train, y_train_encoded, tokenizer, max_length=128)
val_dataset = CustomDataset(X_val, y_val_encoded, tokenizer, max_length=128)
test_dataset = CustomDataset(X_test, y_test_encoded, tokenizer, max_length=128)


In [34]:
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=len(y.unique()))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [35]:
from torch.utils.data import DataLoader
from transformers import AdamW

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

# Huấn luyện mô hình
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_loader)}")



Epoch 1/3, Loss: 1.601935097326835
Epoch 2/3, Loss: 0.4420412291462223
Epoch 3/3, Loss: 0.17092936299741268


In [36]:
from sklearn.metrics import classification_report
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predictions.cpu().numpy())

print(classification_report(y_true, y_pred, target_names=y.unique()))

              precision    recall  f1-score   support

         Why       0.93      0.95      0.94        40
    Quantity       1.00      1.00      1.00        27
      Yes/No       1.00      0.81      0.90        27
         How       0.97      0.97      0.97        34
         Who       0.97      1.00      0.98        31
       Where       1.00      1.00      1.00        33
        What       0.93      1.00      0.96        25
        When       0.92      0.96      0.94        23

    accuracy                           0.96       240
   macro avg       0.96      0.96      0.96       240
weighted avg       0.96      0.96      0.96       240



In [37]:
def predict_question(question, model, tokenizer, label_encoder):
    model.eval()
    encoding = tokenizer(question, return_tensors="pt", truncation=True, padding=True, max_length=50)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=1).cpu().numpy()

    return label_encoder.inverse_transform(prediction)[0]

# Ví dụ dự đoán
new_question = "Tôi muốn biết tổng thống Hàn Quốc bây giờ là ai?"
predicted_label = predict_question(new_question, model, tokenizer, label_encoder)
print(f"Câu hỏi: {new_question}")
print(f"Loại câu hỏi dự đoán: {predicted_label}")


Câu hỏi: Tôi muốn biết tổng thống Hàn Quốc bây giờ là ai?
Loại câu hỏi dự đoán: Who
